# HF Classification of Bonfiglioli's Vibration Signals

Introduction

In [ ]:
# Dependencies

### Get Data

Define first a pytorch dataset able to load numpy files: remember that, because of the function `load_hf`, the preprocessed data are stored in 6 _.npy_ files called `x_train`, `x_val`, `x_test`, `y_train`, `y_val`, `y_test`.

In [ ]:
class NpyDataset(Dataset):
    
    def __init__(self, x_path, y_path, transform=None):
        
        self.x = np.load(x_path)
        self.y = np.load(y_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        
        sample = (self.x[idx], self.y[idx])
        
        if self.transform is not None:
            sample = self.transform(sample)
        
        return sample

In [ ]:
class ToTensor_onehot(object):
    """Convert sample to Tensors."""
    def __call__(self, sample):
        x, y = sample
        # One-hot encoding
        y = np.eye(2)[y]
        return ( torch.from_numpy(x), torch.tensor([y]) )
    
class Squeeze(object):
    """Remove the extra dimensions of the tensors."""
    def __init__(self, squeeze_x=True, squeeze_y=True):
        self.squeeze_x = squeeze_x
        self.squeeze_y = squeeze_y
    
    def __call__(self, sample):
        x, y = sample
        if self.squeeze_x:
            x = torch.squeeze(x)
        if self.squeeze_y:
            y = torch.squeeze(y)
        return ( x, y )
    
class Standardize_Frame(object):
    """Since each row of a sample correspond to a single frame of the spectrum,
    it is possible to standardize them in order to have features centered around 0
    with unitary variance."""
    def __call__(self, sample):
        x, y = sample
        std, mean = torch.std_mean(x, dim=0, unbiased=True, keepdim=True)
        x = (x - mean) / std
        return (x, y)
    
class Normalize(object):
    def __call__(self, sample):
        x, y = sample
        x_norm = np.abs(x)
        x_min, x_max = x_norm.min(), x_norm.max()
        x = (x - x_min) / (x_max - x_min)
        return (x, y)
    
class SwapAxes(object):
    def __call__(self, sample):
        x, y = sample
        return (torch.swapaxes(x, 0, 1), torch.squeeze(y))
    